<a href="https://colab.research.google.com/github/KunaCornejo/SmartGatewaySelection/blob/main/SmartSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy import genfromtxt
from geopy.distance import great_circle

In [2]:
#import git
!git clone https://github.com/KunaCornejo/SmartGatewaySelection.git


Cloning into 'SmartGatewaySelection'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [18]:
ls 'SmartGatewaySelection'

c_n_ul.csv  SmartSelection.ipynb


In [24]:
c_n_ul = genfromtxt('/content/SmartGatewaySelection/c_n_ul.csv', delimiter=',') 
c_n_ul=list(c_n_ul)

In [26]:
ubicaciones=['Panama','San Jose','Tegucigalpa','CDMX','Monterrey','Guadalajara'\
             ,'Tijuana','La Habana','Sto. Domingo','San Salvador','San Juan',\
             'Torreon','La Paz (BC)','Veracruz','Cancun','Queretaro'\
             ,'Tuxtla (Gtz)','Cd. Juarez','San Pedro Sula','Belmopan','Kingston'\
             ,'Merida','Pto. Cabezas','Oaxaca']
ubicaciones=np.transpose([ubicaciones])
C_N_UL=np.array([c_n_ul],dtype=object) 
C_N_UL=np.transpose(C_N_UL)
C_N_UL=np.delete(C_N_UL, 15, 0) #Get rid of Chihuahua, it is not necessary
CN_UP=np.concatenate((ubicaciones,C_N_UL),axis=1) #Concatenamos two lists
print(CN_UP)

[['Panama' 32.4401636]
 ['San Jose' 32.4611016]
 ['Tegucigalpa' 32.4450513]
 ['CDMX' 32.3918091]
 ['Monterrey' 32.313151]
 ['Guadalajara' 32.3588726]
 ['Tijuana' 32.0981607]
 ['La Habana' 32.3380601]
 ['Sto. Domingo' 32.2887155]
 ['San Salvador' 32.4521009]
 ['San Juan' 32.2438352]
 ['Torreon' 32.301041]
 ['La Paz (BC)' 32.2705558]
 ['Veracruz' 32.4018877]
 ['Cancun' 32.3779865]
 ['Queretaro' 32.3736176]
 ['Tuxtla (Gtz)' 32.4294244]
 ['Cd. Juarez' 32.195777]
 ['San Pedro Sula' 32.4354086]
 ['Belmopan' 32.4215586]
 ['Kingston' 32.3572017]
 ['Merida' 32.3851709]
 ['Pto. Cabezas' 32.4314209]
 ['Oaxaca' 32.4205395]]
